In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy import optimize
from scipy.stats import multivariate_normal
import pandas as pd
import os
from math import *

In [2]:
#Parameters
mu=[np.zeros(3),np.array([1,5,-3]),np.zeros(3)]
sig=[np.diag([3,5,2]),np.array([[1,0,0],[0,4,1],[0,1,6]]),10*np.eye(3)]

In [5]:
data_set=[]
for i in range(3):
    data_set.append(np.random.multivariate_normal(mu[i],sig[i],20))

In [101]:
def mean_ml_est(data):
    n,m=data.shape
    mu=sum(data)/n
    return mu
def cov_ml_est(data):
    n,m=data.shape
    x=data-mean_ml_est(data)
    cov_mat=(x.transpose() @ x)/n
    return cov_mat

In [102]:
#Part a
mean_ml_estimates=[]
cov_ml_estimates=[]
for i in range(3):
    mean_ml_estimates.append(mean_ml_est(data_set[i]))
    cov_ml_estimates.append(cov_ml_est(data_set[i]))

In [113]:
def normalize_man(data):
    data_diff=data-np.mean(data,0)
    return data_diff/np.std(data,0)

In [140]:
data_n=[normalize_man(data_set[i]) for i in range(3)]
data_con=np.concatenate((data_n[0],data_n[1],data_n[2]),axis=0)
(cov_n,mu_n)=(cov_ml_est(data_con),mean_ml_est(data_con))

In [131]:
def cov_shrink(cov_i,cov_n,a,n_i,n): #0<a<1
    cov_s=(((1-a)*n_i*cov_i)+(a*n*cov_n))/((1-a)*n_i+a*n)
    return cov_s

In [134]:
#cov_shrink(cov_ml_estimates[0],cov_n,0.1,20,60)

In [166]:
df=pd.DataFrame(data_con,columns=['x1','x2','x3'])
df['y']=[i for i in range(3) for j in range(20)]

In [167]:
#Use cov_shrink for (0,1,2) for different alphas and train classifier using df
# plot the graph
# Now generate another 50 data sets don't train the model plot the test error vs salpha graph after that.